In [1]:
# 基本パッケージ（numpy,Pandas,matplotlib）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# globのimport
import glob

In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)

In [3]:
# 2001～2018年の株価データをマージする
## globでファイル名の一覧を取得
stock_price_files = glob.glob('stockPrice/*.csv')
stock_price_list = []

# ファイルを読み込み、DataFrameでlistに格納する。
for f in stock_price_files:
    stock_price_list.append(pd.read_csv(f, header=1, encoding="shift-jis", parse_dates = [0]))
    
# Listに格納されたデータを全てconcat関数で連結    
stock_price_all = pd.concat(stock_price_list)

# 列名の変更
stock_price_all.rename(columns = {'日付':'Date','始値':'Open','高値':'High','安値':'Low','終値':'Close','終値調整値':'Adj Close',
                                  '出来高':'Volume'}, inplace = True)
# 移動平均（5日）の作成
#stock_price_all['ma_5'] = stock_price_all['Adj Close'].rolling(5).mean()

# EMAの作成
stock_price_all['ema_5'] = stock_price_all['Adj Close'].ewm(span=5).mean()

# MACDの作成
stock_price_all['macd'] = stock_price_all['Adj Close'].ewm(span=5).mean() - stock_price_all['Adj Close'].ewm(span=25).mean()

# ボリンジャーバンドの作成
#stock_price_all['bband_upper'] = stock_price_all['ma'] +(stock_price_all['Adj Close'].rolling(5).std()*2)
#stock_price_all['bband_lower'] = stock_price_all['ma'] -(stock_price_all['Adj Close'].rolling(5).std()*2)

# 移動平均乖離率の作成
creating_feature = pd.DataFrame({'Date' : stock_price_all['Date'] })
#creating_feature['pvma_5'] = (stock_price_all['Adj Close'] - stock_price_all['ma_5'])/stock_price_all['ma_5']
creating_feature['pvema_5'] = (stock_price_all['Adj Close'] - stock_price_all['ema_5'])/stock_price_all['ema_5']


# モメンタム（10日）の作成
#creating_feature['momentum'] = stock_price_all['Adj Close'].diff(10).apply( lambda x: 0 if x < 0 else 1 )

# 移動平均と終値の関係
#creating_feature['cross_5'] =(stock_price_all['Adj Close'] - stock_price_all['ma_5']).apply( lambda x: 0 if x < 0 else 1 )
#creating_feature['cross_25'] =(stock_price_all['Adj Close'] - stock_price_all['ma_25']).apply( lambda x: 0 if x < 0 else 1 )
#creating_feature['cross_ma'] =(stock_price_all['ma_5'] - stock_price_all['ma_25']).apply( lambda x: 0 if x < 0 else 1 )


# 欠損値のチェック
print(stock_price_all.isnull().sum())

# 読み込みデータの確認
stock_price_all.head()

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
ema_5        0
macd         0
dtype: int64


,Date,Open,High,Low,Close,Volume,Adj Close,ema_5,macd
0,2001-07-13,12490,12490,12300,12310,259430,12310,12310.000000,0.000000
1,2001-07-16,12400,12400,12250,12330,99740,12330,12322.000000,1.600000
2,2001-07-17,12170,12170,12100,12120,93150,12120,12226.315789,-21.871844
3,2001-07-18,12150,12150,11830,11840,165330,11840,12065.846154,-67.735955
4,2001-07-19,11990,11990,11860,11920,620220,11920,12009.857820,-73.911543


In [4]:
# creating_featureの確認
creating_feature.head()

,Date,pvema_5
0,2001-07-13,0.000000
1,2001-07-16,0.000649
2,2001-07-17,-0.008696
3,2001-07-18,-0.018718
4,2001-07-19,-0.007482


In [5]:
# 2002～2018年の為替レートを取得
market_Data = pd.read_csv('market/market.csv', header=0, encoding="shift-jis", parse_dates = [0])

# 不要な通貨を削除
#market_Data.drop(columns=['USD','SEK','DKK','NOK','AUD','CNY','HKD','PHP','SGD','KWD','SAR','AED','TWD'], inplace=True)

# 欠損値のチェック
print(market_Data.isnull().sum())

# レート情報をずらす
market_Data = market_Data.shift(-1)

# 読み込みデータの表示
market_Data.head()

Date        0
USD         0
GBP         0
EUR         0
CAD         0
CHF         0
SEK         0
DKK         0
NOK         0
AUD         0
NZD         0
ZAR         0
BHD         0
IDR(100)    0
CNY         0
HKD         0
INR         0
PHP         0
SGD         0
KRW(100)    0
THB         0
KWD         0
SAR         0
AED         0
MXN         0
TWD         0
dtype: int64


,Date,USD,GBP,EUR,CAD,CHF,SEK,DKK,NOK,AUD,NZD,ZAR,BHD,IDR(100),CNY,HKD,INR,PHP,SGD,KRW(100),THB,KWD,SAR,AED,MXN,TWD
0,2002-04-02,133.20,191.78,117.18,83.38,80.15,13.00,15.77,15.24,71.02,58.86,11.78,353.78,1.38,16.09,17.08,2.73,2.62,72.18,10.12,3.06,435.01,35.53,36.27,14.80,3.82
1,2002-04-03,133.20,191.26,116.96,83.65,80.02,12.95,15.75,15.23,71.14,58.81,11.89,353.78,1.37,16.09,17.08,2.74,2.62,72.12,10.03,3.04,436.58,35.53,36.27,14.77,3.82
2,2002-04-04,133.10,191.13,117.15,83.72,80.18,12.93,15.77,15.33,70.81,58.52,12.02,353.52,1.37,16.08,17.06,2.73,2.61,72.26,10.02,3.05,435.11,35.50,36.24,14.75,3.82
3,2002-04-05,132.30,189.74,116.32,82.96,79.47,12.87,15.65,15.22,70.33,58.36,11.83,351.39,1.37,15.98,16.96,2.71,2.60,71.85,10.03,3.04,432.21,35.29,36.03,14.66,3.80
4,2002-04-08,131.55,188.54,115.69,82.80,79.00,12.82,15.57,15.13,69.76,57.43,11.92,349.40,1.38,15.89,16.87,2.70,2.58,71.76,9.90,3.03,429.62,35.09,35.81,14.62,3.77


In [6]:
# 2002～2018年の主要指数データをマージする
## globでファイル名の一覧を取得
world_indices_files = glob.glob('worldIndices/*.csv')

# ファイルを読み込み、DataFrameでlistに格納する。
for i in range(len(world_indices_files)):
    if i == 0:
         world_indices_data = pd.read_csv(world_indices_files[i], header=0, encoding='shift-jis', parse_dates = [0])
         world_indices_data.drop(columns=['Close','Volume'], inplace=True)
         world_indices_data.rename(columns = {'Open':'Open'+ world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'High':'High'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Low':'Low'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Adj Close':'Adj Close'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Volume':'Volume'+world_indices_files[i].replace('worldIndices\\','').replace('.csv','')}, inplace = True)
    else:      
        work_data = pd.read_csv(world_indices_files[i], header=0, encoding='shift-jis', parse_dates = [0])
        work_data.drop(columns=['Close','Volume'], inplace=True)
        work_data.rename(columns = {'Open':'Open'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'High':'High'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Low':'Low'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Adj Close':'Adj Close'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Volume':'Volume'+world_indices_files[i].replace('worldIndices\\','').replace('.csv','')}, inplace = True)
        world_indices_data = pd.merge(world_indices_data, work_data, on='Date', how='left')

# 欠損値のチェック
print(world_indices_data.isnull().sum())

# 欠損値の補完
world_indices_data = world_indices_data.fillna(method='ffill')

# 読み込みデータの表示
world_indices_data.head()

Date                    0
Open000001.SS           2
High000001.SS           2
Low000001.SS            2
Adj Close000001.SS      2
Open^AORD             463
High^AORD             463
Low^AORD              463
Adj Close^AORD        463
Open^AXJO             465
High^AXJO             465
Low^AXJO              465
Adj Close^AXJO        465
Open^BFX               63
High^BFX               63
Low^BFX                63
Adj Close^BFX          63
Open^BSESN            200
High^BSESN            200
Low^BSESN             200
Adj Close^BSESN       200
Open^BVSP             184
High^BVSP             184
Low^BVSP              184
Adj Close^BVSP        184
Open^DJI              144
High^DJI              144
Low^DJI               144
Adj Close^DJI         144
Open^FCHI              64
                     ... 
Low^MERV              231
Adj Close^MERV        231
Open^MXX              139
High^MXX              139
Low^MXX               139
Adj Close^MXX         139
Open^N100              64
High^N100   

,Date,Open000001.SS,High000001.SS,Low000001.SS,Adj Close000001.SS,Open^AORD,High^AORD,Low^AORD,Adj Close^AORD,Open^AXJO,High^AXJO,Low^AXJO,Adj Close^AXJO,Open^BFX,High^BFX,Low^BFX,Adj Close^BFX,Open^BSESN,High^BSESN,Low^BSESN,Adj Close^BSESN,Open^BVSP,High^BVSP,Low^BVSP,Adj Close^BVSP,Open^DJI,High^DJI,Low^DJI,Adj Close^DJI,Open^FCHI,High^FCHI,Low^FCHI,Adj Close^FCHI,Open^GDAXI,High^GDAXI,Low^GDAXI,Adj Close^GDAXI,Open^GSPC,High^GSPC,Low^GSPC,Adj Close^GSPC,Open^GSPTSE,High^GSPTSE,Low^GSPTSE,Adj Close^GSPTSE,Open^HSI,High^HSI,Low^HSI,Adj Close^HSI,Open^IPSA,High^IPSA,Low^IPSA,Adj Close^IPSA,Open^IXIC,High^IXIC,Low^IXIC,Adj Close^IXIC,Open^JKSE,High^JKSE,Low^JKSE,Adj Close^JKSE,Open^KS11,High^KS11,Low^KS11,Adj Close^KS11,Open^MERV,High^MERV,Low^MERV,Adj Close^MERV,Open^MXX,High^MXX,Low^MXX,Adj Close^MXX,Open^N100,High^N100,Low^N100,Adj Close^N100,Open^NYA,High^NYA,Low^NYA,Adj Close^NYA,Open^RUT,High^RUT,Low^RUT,Adj Close^RUT,Open^TWII,High^TWII,Low^TWII,Adj Close^TWII,Open^VIX,High^VIX,Low^VIX,Adj Close^VIX,Open^XAX,High^XAX,Low^XAX,Adj Close^XAX
0,2002-04-01,1598.026001,1609.916992,1589.406982,1608.505981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3482.939941,3519.320068,3482.939941,3500.179932,13256.0,13499.0,13068.0,13467.0,10402.070313,10402.070313,10263.679688,10362.700195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1147.390015,1147.839966,1132.869995,1146.540039,7854.000000,7932.100098,7850.000000,7888.500000,NaN,NaN,NaN,NaN,1163.630005,1164.579956,1149.910034,1151.329956,1834.589966,1865.369995,1817.250000,1862.619995,482.524994,486.667999,481.006989,486.653564,900.469971,900.469971,874.900024,875.830017,NaN,NaN,NaN,NaN,7362.140137,7427.160156,7341.350098,7371.890137,NaN,NaN,NaN,NaN,6327.109863,6327.109863,6327.109863,6327.109863,506.459991,506.459991,498.399994,504.500000,6118.450195,6213.319824,6118.450195,6186.416504,18.990000,19.500000,18.610001,18.730000,910.489990,915.210022,910.270020,914.750000
1,2002-04-02,1608.519043,1610.727051,1575.972046,1582.883057,3363.300049,3363.300049,3332.399902,3338.500000,3414.800049,3418.000000,3380.199951,3387.300049,2766.360107,2766.360107,2740.689941,2755.050049,3505.399902,3537.290039,3490.870117,3505.790039,13482.0,13535.0,13245.0,13245.0,10352.459961,10352.459961,10264.860352,10313.709961,4673.029785,4688.160156,4626.459961,4627.330078,5379.640137,5379.640137,5289.799805,5311.080078,1146.540039,1146.540039,1135.709961,1136.760010,7895.899902,7920.100098,7886.700195,7893.399902,11032.360352,11055.980469,10816.730469,10878.040039,1150.619995,1151.089966,1139.849976,1139.969971,1836.030029,1839.369995,1804.400024,1804.400024,487.178009,491.790009,485.949005,489.074493,878.900024,905.340027,871.450012,905.340027,436.220001,447.399994,432.750000,435.089996,7370.149902,7387.310059,7316.689941,7316.689941,817.280029,819.659973,812.359985,812.849976,6305.330078,6305.330078,6305.330078,6305.330078,504.500000,504.500000,500.489990,500.489990,6248.040039,6326.129883,6230.350098,6243.436523,19.120001,19.410000,19.120001,19.160000,914.750000,919.669983,913.880005,915.849976
2,2002-04-03,1582.744019,1603.797974,1582.671997,1594.508057,3338.899902,3343.100098,3329.000000,3329.000000,3387.300049,3393.300049,3377.800049,3377.800049,2755.110107,2784.169922,2748.649902,2783.360107,3500.760010,3500.760010,3458.850098,3462.989990,13280.0,13345.0,13061.0,13085.0,10311.809570,10339.860352,10139.480469,10198.290039,4618.649902,4644.209961,4582.709961,4632.259766,5293.370117,5322.549805,5266.060059,5281.839844,1136.760010,1138.849976,1119.680054,1125.400024,7869.000000,7887.100098,7832.000000,7842.299805,10831.230469,10853.879883,10783.000000,10833.959961,1139.969971,1146.709961,1139.729980,1146.359985,1809.589966,1813.359985,1770.609985,1784.349976,489.048004,492.079987,487.933990,488.142517,899.979980,919.890015,898.280029,918.590027,435.089996,435.089996,426.390015,431.399994,7316.589844,7319.600098,7191.750000,7191.939941,812.070007,813.369995,806.469971,811.590027,6245.689941,6245.689941,6245.689941,624

In [7]:
# 内部結合で結合する
merge_work= pd.merge(world_indices_data,market_Data, on='Date', how='inner')
merge_Data = pd.merge(stock_price_all, merge_work, on='Date', how='inner')

In [8]:
# 目的変数の作成
merge_Data['diff'] = merge_Data['Close'] - merge_Data['Open']
answersData = pd.DataFrame({'Date' : merge_Data['Date']})
answersData['answer'] =merge_Data['diff'].apply( lambda x: 0 if x < 0 else 1 )

# 不要な列を削除
merge_Data.drop(columns=['Date','Close','diff'], inplace=True)

# 各列を変化率へ一括変換
merge_Data_pct = merge_Data.pct_change()

# 作製した特徴量と目的変数の付与
make_data = pd.merge(creating_feature, answersData, on='Date', how='inner')
make_data.drop(columns=['Date'], inplace=True)
stock_Price_Prediction = pd.concat([merge_Data_pct, make_data], axis=1)


# 正解ラベルをずらし、説明変数から見て未来の値とする
stock_Price_Prediction.answer = stock_Price_Prediction.answer.shift(-1)


In [9]:
# 欠損値のチェック
print(stock_Price_Prediction.isnull().sum())

# NaN,INF,-INFは削除する
stock_Price_Prediction = stock_Price_Prediction.replace([np.inf, -np.inf], np.nan)
stock_Price_Prediction = stock_Price_Prediction.dropna()

# CSVへ変換
stock_Price_Prediction.to_csv("stock_Price_Prediction_v3.3.csv",index = False, encoding="shift-jis")

Open                  1
High                  1
Low                   1
Volume                1
Adj Close             1
ema_5                 1
macd                  1
Open000001.SS         1
High000001.SS         1
Low000001.SS          1
Adj Close000001.SS    1
Open^AORD             1
High^AORD             1
Low^AORD              1
Adj Close^AORD        1
Open^AXJO             1
High^AXJO             1
Low^AXJO              1
Adj Close^AXJO        1
Open^BFX              1
High^BFX              1
Low^BFX               1
Adj Close^BFX         1
Open^BSESN            1
High^BSESN            1
Low^BSESN             1
Adj Close^BSESN       1
Open^BVSP             1
High^BVSP             1
Low^BVSP              1
                     ..
High^XAX              1
Low^XAX               1
Adj Close^XAX         1
USD                   1
GBP                   1
EUR                   1
CAD                   1
CHF                   1
SEK                   1
DKK                   1
NOK             